## Traditional machine learning
Feature(s): 'SMILES' column

- 'SMILES' column

Target: deltaG4_bin

- Binary class

In [1]:
import sys
import os

# Append the parent directory of your package to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..')))

In [2]:
import pandas as pd
import numpy as np
import zipfile

path_to_dataset = 'NiCatalysis.csv'
csv_filename = 'NiCatalysis.csv'

# Open the file, Correct the encoding and sep if necessary
if path_to_dataset.endswith('.zip'):
    with zipfile.ZipFile(path_to_dataset, 'r') as z:
        # Open the CSV file within the ZIP file
        with z.open(csv_filename) as f:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(f, sep=',', on_bad_lines='warn', index_col = 0)
else:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(path_to_dataset, sep=',', on_bad_lines='warn', index_col = 0)

df = df.rename(columns={'Smiles': 'SMILES'})

print('Count of unique smiles:', df.SMILES.unique().shape[0])
print('Count of all of the smiles:', df.shape[0])


Count of unique smiles: 1423
Count of all of the smiles: 1423


In [3]:
display(df.head())
df.shape

,Unnamed: 0.1,Unnamed: 0,Name,SMILES,\DeltaG4,type,deltaG4_bin,deltaG4_realbin-27_-39
0,0,0,Carb_0_0_10,*[Ni@](C9=N([H])C(C(C)C)=C(C(C)C)N([H])9)(c1cc...,-19.000359,Carb,0,0
1,1,1,Carb_0_0_1,*[Ni@](C9=N([H])C(C)C(C)N([H])9)(c1ccc2ccccc2c...,-8.308853,Carb,1,0
2,2,2,Carb_0_0_25,*[Ni@](C9=N([H])C1C2=CC=CC(=C23)C=CC=C3C=1N([H...,-9.688746,Carb,1,0
3,3,3,Carb_0_0_7,*[Ni@](C9=N([H])C(C(F)(F)F)C(C(F)(F)F)N([H])9)...,-10.292410,Carb,1,0
4,4,4,Carb_0_11_0,*[Ni@](C9=N([H])CCN(C1CC1)9)(c1ccc2ccccc2c1)([...,-5.979538,Carb,1,0


(1423, 8)

In [4]:
from MLPipeline import MLmodel, BinTheTarget

Target = ['deltaG4_bin']
Features = ['SMILES']
Feature_types = ['SMILES']
input = df

/home/lsmo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = MLmodel(modelType='RandomForestClassifier',
                    df=input,
                    target=Target,
                    features=Features,
                    feature_types=Feature_types)

# get the values (input and output) of the model
X_train, X_test, y_train, y_test = model.getValues()

model.train()
model.evaluate()

[17:54:32] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:32] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:32] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:32] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:32] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:32] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:32] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:32] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:32] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:32] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:32] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:32] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:32] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:32] Explicit valence for atom # 3 N, 4, is g

RandomForestClassifier model trained successfully.
Accuracy for RandomForestClassifier: 0.90


0.9

In [6]:
number_of_samples, number_of_wrong_smiles, clean_df = model.getdfAnalysis(orginal_df=input)
print('Number of samples:', number_of_samples)
print('Number of wrong smiles:', number_of_wrong_smiles)


for i in range(len(clean_df[Target].value_counts())):
    print(f'bin {i} fraction in original set:' , 
          clean_df[Target].value_counts()[i]/clean_df[Target].value_counts().sum())

for i in range (len(pd.DataFrame(y_train).value_counts())):
    print(f'bin {i} fraction in training set:' , 
          pd.DataFrame(y_train).value_counts()[i]/pd.DataFrame(y_train).value_counts().sum())

for i in range(len(pd.DataFrame(y_test).value_counts())):
    print(f'bin {i} fraction in test set:' , 
          pd.DataFrame(y_test).value_counts()[i]/pd.DataFrame(y_test).value_counts().sum())

Number of samples: 1423
Number of wrong smiles: 770
bin 0 fraction in original set: 0.9157733537519143
bin 1 fraction in original set: 0.08422664624808576
bin 0 fraction in training set: 0.88
bin 1 fraction in training set: 0.12
bin 0 fraction in test set: 0.9
bin 1 fraction in test set: 0.1


In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.base import clone

def objective(trial, model_instance):
    """
    Objective function for Optuna to minimize.
    """
    # Define hyperparameters to tune
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_categorical('max_depth', [None, 10, 20, 30, 40]),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 15),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 6),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    # Clone the model to ensure a fresh instance each trial
    model_clone = clone(model_instance.model)
    model_clone.set_params(**params)
    
    # Define the score metric
    scoring = 'accuracy'

    # Perform cross-validation
    scores = cross_val_score(model_clone, model_instance.X_train, model_instance.y_train, cv=model_instance.cv, scoring=scoring)

    # Return the average score across all folds
    return scores.mean()

In [8]:
model = MLmodel(modelType='RandomForestClassifier', df=input, target=Target, 
                features=['SMILES'], hyperparameter_tuning=True,
                feature_types=Feature_types,
                optimization_method='optuna', objective=lambda trial: objective(trial, model))

model.train()
predictions = model.predict()
model.evaluate()

[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is g

[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is greater than permitted
[17:54:33] Explicit valence for atom # 3 N, 4, is g

Best RandomForestClassifier model trained successfully with hyperparameter tuning using Optuna.
Best hyperparameters: {'n_estimators': 151, 'max_depth': 40, 'min_samples_split': 9, 'min_samples_leaf': 3, 'max_features': 'log2', 'bootstrap': False}
RandomForestClassifier model trained successfully.
Accuracy for RandomForestClassifier: 0.90


0.9